In [12]:
import argparse
import json
from datetime import datetime, timedelta, date
from pathlib import Path
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from dateutil.relativedelta import *
#from pycoingecko import CoinGeckoAPI
from tqdm import tqdm
from pandas import read_csv



def get_token_data(token, api_key, end_date, start_date, put_date_info=True):
    params = {
        "format": "csv",
        "key": api_key,
        "from": start_date,
        "to": end_date,
    }
    url = f"https://api.covalenthq.com/v1/pricing/historical/usd/{token}/"
    data_history_response = requests.get(url=url, params=params)
    data_history_response = data_history_response.text.split('\n')
    if len(data_history_response) < 365:
        print(f'Exclude {token} token, because of unavailability of its data!')
        return None

    data_history_response = data_history_response[:-1] # remove the last item (it's empty)
    headers = data_history_response[0]
    data_history_response = data_history_response[1:] # remove the header
    if put_date_info:
        date_info = [[x.split(',')[0].strip('"'), x.split(',')[1].strip('"'), None] for x in data_history_response] # remove redundant double quote
    else:
        date_info = [[x.split(',')[1].strip('"'), None] for x in data_history_response] # remove redundant double quote

    return date_info



def get_historical_data(token, api_key, end_date, window):
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d").date()
    start_date_dt = end_date_dt - timedelta(days=window)
    start_date = start_date_dt.strftime("%Y-%m-%d")
    print(f"start date: {start_date}, end date: {end_date}")
    data_history_response = get_token_data(token, api_key, end_date, start_date)
    new_headers = ["Date", "Price", "Token", "Symbol"]
    df = pd.DataFrame(data_history_response, columns=new_headers)
    print(f"The length of the received data is: {len(data_history_response)}")  # Print the url
    print(f"Just show the head of dataframe:\n{df.head()}")

    return df



def get_tokens_history(tokens, api_key, end_date, window):
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d").date()
    start_date_dt = end_date_dt - timedelta(days=window)
    start_date = start_date_dt.strftime("%Y-%m-%d")
    print(f"start date: {start_date}, end date: {end_date}, window: {window}")
    excluded_tokens = []
    headers = ["date"]
    data = []
    for idx, token in enumerate(tqdm(tokens)):
        data_history = get_token_data(token, api_key, end_date, start_date, True if idx == 0 else False)
        if data_history is None:
            excluded_tokens.append(token)
            continue

        if idx == 0:
            data = data_history
        else:
            data = [a + b for a, b in zip(data, data_history)]

        headers += [token, token + "_priceChange"]
        

    df = pd.DataFrame(data, columns=headers)
    df = df.sort_values(by='date', ascending = True)
    tokens = [token for token in tokens if token not in excluded_tokens]
    
    for i in range(df.index.size):
        if not i:
            continue

        for token in tokens:
            # Calculate price chnage relative to 1 
            df[token + "_priceChange"].iloc[i] = str(((float(df[token].iloc[i]) - float(df[token].iloc[i-1])) / float(df[token].iloc[i-1])) + 1)
            
            
    df.drop(0, inplace=True)

    return df


def main(args):
    tokens = args.tokens
    chain = args.chain
    api_key = args.api_key
    window = args.window
    end_date = args.end_date
    result_dir = Path(args.result_dir)
    result_dir.mkdir(exist_ok=True)

    NOW = datetime.now()
    today_date = NOW.date()
    today_time_str = NOW.strftime("%H-%M-%S")
    print('time:', today_time_str)

    data_df = get_tokens_history(tokens, api_key, end_date, window)
    file_name = result_dir / "result.csv"
    if file_name.is_file():
        file_name.unlink()

    data_df.to_csv(file_name, sep=',', encoding='utf-8')
 

today = str(date.today())


if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Fetch data from Covalent")
    parser.add_argument("--chain",
                        default="mainnet",
                        choices=["mainnet", "rinkeby"],
                        help="visualize train and test data")
    parser.add_argument("--api_key",
                        default="ckey_INSERT_YOUR_KEY_HERE",
                        help="your api key on Covalent")
    parser.add_argument("--tokens",
                        default= ['aave', 'bal', 'link', 'crv', 'dai', 'gno', 'mke', 'rai', 'grt', 'uni', 'yfi'],
                        help="list of tokens")
    parser.add_argument("--result_dir",
                        default="./result",
                        help="the result directory path")
    parser.add_argument("--end_date",
                        default=today,
                        help="end date of history data")
    parser.add_argument("--window",
                        default=365,
                        help="the length of hisotry data to fetch")

    args, unknown = parser.parse_known_args()
    main(args)


results = pd.read_csv('result/result.csv', sep=',')
results = results.iloc[: , 1:]    # Removes extra index
results = results.sort_values(by='date', ascending = False)
results = results[:-1]            # Removes null row


# Create DF for SIPmath Library 
results1 = pd.DataFrame(results['date'])
results2 = results.filter(like='_priceChange')
resultsDF = pd.concat([results1, results2], axis=1)
resultsDF.to_csv("./result/resultDF.csv", sep=',', encoding='utf-8')

resultsDF

time: 23-22-06
start date: 2021-04-03, end date: 2022-04-03, window: 365


 64%|██████▎   | 7/11 [00:01<00:01,  3.47it/s]

Exclude mke token, because of unavailability of its data!


100%|██████████| 11/11 [00:03<00:00,  3.58it/s]


,date,aave_priceChange,bal_priceChange,link_priceChange,crv_priceChange,dai_priceChange,gno_priceChange,rai_priceChange,grt_priceChange,uni_priceChange,yfi_priceChange
364,2022-04-02,0.991849,0.978164,1.039151,1.067326,1.019111,0.997142,1.016293,1.016293,1.043494,1.020749
363,2022-04-01,1.136231,0.998957,0.971422,1.012718,0.955464,0.984403,0.958600,0.967385,0.975385,0.998761
362,2022-03-31,0.944303,0.958907,1.006424,0.953410,1.024869,0.924013,1.020384,0.991658,0.985259,0.950196
361,2022-03-30,1.050498,1.166376,1.036152,1.025741,1.016595,0.997866,1.017009,1.009882,1.065211,1.052945
360,2022-03-29,1.277891,1.040870,1.020173,1.079838,1.002182,1.040465,1.002060,1.005989,1.016300,1.039466
...,...,...,...,...,...,...,...,...,...,...,...
5,2021-04-08,0.990753,0.875261,0.981076,0.980370,0.942105,0.968770,0.974290,0.936726,0.987758,1.056442
4,2021-04-07,0.931800,0.969949,0.918759,0.930084,1.015788,0.958771,0.985482,1.050640,0.932658,1.081128
3,2021-04-06,0.989451,1.004000,1.095519,0.979824,1.023019,1.015533,1.022648,1.018253,1.039682,1.023620
2,2021-04-05,0.982603,1.046750,1.046868,1.093549,1.008624,1.025083,1.013111,1.380267,1.023405,1.054807
